In [3]:
"""
Feature-engineering + daily cash-flow aggregation
=================================================
Adds on top of your existing script:
    • daily income / expenses / net_cash_flow
    • one-row-per-day feature snapshot
    • gap-filled calendar index per user (zeros for quiet days)
2025-06-19
"""

import pandas as pd
import numpy as np
from pathlib import Path

# ------------------------------------------------------------
# 0 · PATHS — adjust only these two lines if the file lives elsewhere
# ------------------------------------------------------------
CSV_PATH         = Path(r"C:/Users/loics/OneDrive/Documents/1. BAM/BLOCK 5/Assignment coding/synthetic_transactions.csv")
SAVE_TX_PATH     = Path(r"C:/Users/loics/OneDrive/Documents/1. BAM/BLOCK 5/Assignment coding/synthetic_transactions_enriched.csv")
SAVE_DAILY_PATH  = Path(r"C:/Users/loics/OneDrive/Documents/1. BAM/BLOCK 5/Assignment coding/synthetic_transactions_daily_enriched.csv")

# ------------------------------------------------------------
# 1 · LOAD  +  ROBUST DATE PARSING
# ------------------------------------------------------------
df = pd.read_csv(CSV_PATH)
df["date"] = pd.to_datetime(df["date"], dayfirst=True, errors="raise")
df = df.sort_values(["user_id", "date"]).reset_index(drop=True)

# ------------------------------------------------------------
# 2 · ROLLING USER-LEVEL STATISTICS (7 / 30 / 90 days)
# ------------------------------------------------------------
WINDOWS = [7, 30, 90]
for w in WINDOWS:
    rolled = (
        df.set_index("date")
          .groupby("user_id")["amount"]
          .rolling(f"{w}D", min_periods=1)
    )
    df[f"rolling_{w}d_sum"]  = rolled.sum().values
    df[f"rolling_{w}d_mean"] = rolled.mean().values
    df[f"rolling_{w}d_std"]  = rolled.std(ddof=0).fillna(0).values

# ------------------------------------------------------------
# 3 · DAYS SINCE LAST TRANSACTION
# ------------------------------------------------------------
df["days_since_last_tx"] = (
    df.groupby("user_id")["date"]
      .diff()
      .dt.days
      .fillna(-1)      # –1 marks “no previous tx”
)

# ------------------------------------------------------------
# 4 · CUMULATIVE MONTHLY SPEND  (DEBITS ONLY)
# ------------------------------------------------------------
df["year_month"]   = df["date"].dt.to_period("M")
mask_spend         = df["amount"] < 0             # debits
df["cum_month_spend"] = 0.0
df.loc[mask_spend, "cum_month_spend"] = (
    df[mask_spend]
      .groupby(["user_id", "year_month"])["amount"]
      .cumsum()
      .values
)

# ------------------------------------------------------------
# 5 · TIME SINCE LAST PAYDAY  (+ flag)
# ------------------------------------------------------------
def is_salary(row, threshold=1_000):
    """Proxy: positive inflow above a threshold."""
    return row["amount"] > threshold

df["is_payday"] = df.apply(is_salary, axis=1)
df["time_since_payday"] = (
    df.groupby("user_id")["date"]
      .apply(lambda s: s - s.where(df.loc[s.index, "is_payday"]).ffill())
      .dt.days
      .fillna(-1)           # –1 means “no payday yet”
      .reset_index(level=0, drop=True)
)

# ------------------------------------------------------------
# 6 · SPENDING-CATEGORY RATIOS  (user × month)
# ------------------------------------------------------------
spend_month_tot = (
    df[mask_spend]
      .groupby(["user_id", "year_month"])["amount"]
      .sum()
      .abs()
      .rename("month_total_abs")
)
spend_month_cat = (
    df[mask_spend]
      .groupby(["user_id", "year_month", "category"])["amount"]
      .sum()
      .abs()
      .rename("cat_total_abs")
      .reset_index()
      .merge(spend_month_tot, on=["user_id", "year_month"])
)
spend_month_cat["cat_ratio"] = (
    spend_month_cat["cat_total_abs"] / spend_month_cat["month_total_abs"]
)
ratio_wide = (
    spend_month_cat
      .pivot(index=["user_id", "year_month"],
             columns="category",
             values="cat_ratio")
      .fillna(0)
      .add_prefix("ratio_")
      .reset_index()
)
df = (
    df.merge(ratio_wide,
             on=["user_id", "year_month"],
             how="left")
      .fillna(0)
)

# ------------------------------------------------------------
# 7 · CALENDAR FEATURE
# ------------------------------------------------------------
df["day_of_month"] = df["date"].dt.day

# ------------------------------------------------------------
# 8 · SAVE TRANSACTION-LEVEL FILE
# ------------------------------------------------------------
df.drop(columns=["year_month"], inplace=True)
df.to_csv(SAVE_TX_PATH, index=False)
print(f"✔ Transaction-level file saved → {SAVE_TX_PATH}")

# ====================================================================
#             DAILY  AGGREGATION & FEATURE SNAPSHOT
# ====================================================================

# ------------------------------------------------------------
# 9 · DAILY AGGREGATES (income / expenses / net_cash_flow)
# ------------------------------------------------------------
df["day"] = df["date"].dt.normalize()

daily_cash = (
    df.groupby(["user_id", "day"])["amount"]
      .agg(income=lambda x: x[x > 0].sum(),
           expenses=lambda x: x[x < 0].sum(),
           net_cash_flow="sum")
      .reset_index()
)

# ------------------------------------------------------------
# 10 · DAILY FEATURE SNAPSHOT  (last tx of the day)
#      >>> EXCLUDE 'user_id' <<<
# ------------------------------------------------------------
feature_cols = [
    c for c in df.columns
    if c not in {"user_id", "date", "day", "amount", "description", "category"}
]

daily_feat = (
    df.sort_values(["user_id", "date"])
      .groupby(["user_id", "day"])[feature_cols]
      .last()                     # take final state of the day
      .reset_index()
)

# Merge cash + features
daily = daily_cash.merge(daily_feat, on=["user_id", "day"], how="left")

# ------------------------------------------------------------
# 11 · INSERT MISSING DAYS & FILL GAPS
# ------------------------------------------------------------
user_ids = daily["user_id"].unique()
all_days = pd.date_range(df["date"].min().normalize(),
                         df["date"].max().normalize(),
                         freq="D")

idx = pd.MultiIndex.from_product([user_ids, all_days],
                                 names=["user_id", "day"])
daily = (
    daily.set_index(["user_id", "day"])
         .reindex(idx, fill_value=0)      # 0 for cash cols on quiet days
         .reset_index()
)

# Forward-fill feature columns within each user
daily[feature_cols] = (
    daily.groupby("user_id")[feature_cols]
         .apply(lambda g: g.sort_index().ffill().bfill())
         .values
)

# Rename back to a single date column
daily = (daily.rename(columns={"day": "date"})
               .sort_values(["user_id", "date"])
               .reset_index(drop=True))

# ------------------------------------------------------------
# 12 · SAVE DAILY-LEVEL FILE
# ------------------------------------------------------------
daily.to_csv(SAVE_DAILY_PATH, index=False)
print(f"✔ Daily-level file saved → {SAVE_DAILY_PATH}")



✔ Transaction-level file saved → C:\Users\loics\OneDrive\Documents\1. BAM\BLOCK 5\Assignment coding\synthetic_transactions_enriched.csv
✔ Daily-level file saved → C:\Users\loics\OneDrive\Documents\1. BAM\BLOCK 5\Assignment coding\synthetic_transactions_daily_enriched.csv
